In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from datetime import date
import config

#open Chrome
def open_chrome():
    global driver
    driver = webdriver.Chrome()

#open Helo webpage
def open_helo_webpage(driver):
    driver.get(r'https://helo-frontend-staging-react.azurewebsites.net/containers?viewState=%7B%22queries%22%3A%5B%7B%22queryId%22%3A%22containers_v2%22%2C%22queryName%22%3A%22containers_v2%22%2C%22filters%22%3A%5B%7B%22columnId%22%3A%22issuedBy%22%2C%22operator%22%3A%22AND%22%2C%22condition%22%3A%22blank%22%2C%22filterValueType%22%3A%22text%22%7D%5D%2C%22sort%22%3A%5B%5D%7D%5D%7D')

#login to Helo using "username" and "password"
def login(driver):
    driver.find_element(By.XPATH, '//*[@id="email"]').send_keys(config.username)
    driver.find_element(By.ID, "password").send_keys(config.password)
    driver.find_element(By.ID, "btn-login").click()
    print("Logged in")

#choose the customer
def choose_customer(driver,customer):
    driver.find_element(By.XPATH, '//*[@id="menu-button-:r2n:"]').click()
    driver.find_element(By.XPATH, '//*[@id="react-select-3-input"]').send_keys(customer)
    driver.find_element(By.XPATH, '//*[@id="react-select-3-input"]').send_keys('\ue007')
    print(f"Customer: {customer}")

#change to Helo2
def change_to_helo2(driver):
    driver.find_element(By.XPATH, '//*[@id="menu-button-:r3j:"]').click()
    driver.find_element(By.XPATH, '//*[@id="menu-list-:r3j:-menuitem-:r3p:"]').click()
    driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[2]/div[2]/main/div/div[1]/div[2]/div/div/div/div/div/div/div/div/div/label').click()
    print("Changed to Helo2")

# read the customer smoke test file
def read_customer_smoke_test_file(customer):
    global smoke_test_df
    if customer == "recreational":
        smoke_test_df = pd.read_excel("HELO2 - SMOKE TEST.xlsx", sheet_name=1)
    elif customer == "owens":
        smoke_test_df = pd.read_excel("HELO2 - SMOKE TEST.xlsx", sheet_name=0)
    elif customer == "parker":
        smoke_test_df = pd.read_excel("HELO2 - SMOKE TEST.xlsx", sheet_name=2)
    print("File uploaded")
    return smoke_test_df

#insert the date column
def insert_date_column(smoke_test_df):
    smoke_test_df.insert(3, date.today(), "-")
    print(f"Colunm: {date.today()} inserted")

#run the smoke test for the chosen customer
def smoke_test(smoke_test_df):
    print("Starting smoke test")
    for i, link in enumerate(smoke_test_df["HELO2 - Staging Link"]):
        try: 
            if link.startswith("https"):
                driver.get(link)
                try:
                    while driver.find_element(By.CLASS_NAME, 'ag-loading-text').text == "Loading":
                        time.sleep(1)
                except:
                    pass
                time.sleep(2)
                try:
                    record = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[2]/div[2]/main/div/div[1]/div[2]/div/div/div/div/div/div[1]/div[2]/span').text
                    record = record.split(" ")
                    if record[0].isdecimal() == False:
                        record[0] = record[0].replace(",","")
                    smoke_test_df.iloc[i, 3] = int(record[0])
                    print(record[0])
                except:
                    record0 = 0
                    smoke_test_df.iloc[i, 3] = record0
                    print(record0)
                smoke_test_df.iloc[i, 1] =  smoke_test_df.iloc[i, 3] -  smoke_test_df.iloc[i, 4]
        except:
            pass

#save smoke test
def save_customer_test(customer, smoke_test_df):
    if customer == "recreational":
        global smoke_test_recreational
        smoke_test_recreational = smoke_test_df
        #smoke_test_df.to_excel('HELO2_SMOKE_TEST-REI.xlsx', index=False)
    elif customer == "owens":
        global smoke_test_owens
        smoke_test_owens = smoke_test_df
        #smoke_test_df.to_excel('HELO2_SMOKE_TEST-O&M.xlsx', index=False)
    elif customer == "parker":
        global smoke_test_parker
        smoke_test_parker = smoke_test_df
        #smoke_test_df.to_excel('HELO2_SMOKE_TEST-PH.xlsx', index=False)
    print("File saved")

def close_chrome(customer):
    print(f"{customer} Smoke Test is Done!")
    driver.close()
        
#function to run the smoke test to the chosen customer "recreational", "owens" or "parker"
def smoke_test_by_customer(customer):
    open_chrome()
    open_helo_webpage(driver)
    time.sleep(10)
    login(driver)
    time.sleep(5)
    choose_customer(driver,customer)
    time.sleep(5)
    change_to_helo2(driver)
    time.sleep(2)
    read_customer_smoke_test_file(customer)
    time.sleep(2)
    insert_date_column(smoke_test_df)
    time.sleep(2)
    smoke_test(smoke_test_df)
    time.sleep(2)
    save_customer_test(customer, smoke_test_df)
    time.sleep(2)
    close_chrome(customer)   

#save the excel file with the smoke test of all customers
def save_excel_file():
    with pd.ExcelWriter('helo2_smoke_test_python.xlsx') as writer:  # doctest: +SKIP
        smoke_test_owens.to_excel(writer, sheet_name='O&M - SMOKE TEST', index=False)
        smoke_test_recreational.to_excel(writer, sheet_name='REI - SMOKE TEST', index=False)
        smoke_test_parker.to_excel(writer, sheet_name='PH - SMOKE TEST', index=False)
    
#function to run the smoke test
def run_smoke_test():
    customer_list = ["owens", "recreational", "parker"]
    for customer in customer_list:
        #run the smoke test for the chosen customer: "recreational", "owens" or "parker"
        smoke_test_by_customer(customer)
    save_excel_file()
    print("Smoke Test is finished")
    

run_smoke_test()   


Logged in


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="menu-button-:r2n:"]"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CA6A9642+30946]
	(No symbol) [0x00007FF7CA65E3D9]
	(No symbol) [0x00007FF7CA556FDA]
	(No symbol) [0x00007FF7CA5A822C]
	(No symbol) [0x00007FF7CA5A850C]
	(No symbol) [0x00007FF7CA5EDCB7]
	(No symbol) [0x00007FF7CA5CCAAF]
	(No symbol) [0x00007FF7CA5EB041]
	(No symbol) [0x00007FF7CA5CC813]
	(No symbol) [0x00007FF7CA59A6E5]
	(No symbol) [0x00007FF7CA59B021]
	GetHandleVerifier [0x00007FF7CA7DF84D+1301229]
	GetHandleVerifier [0x00007FF7CA7EBDC7+1351783]
	GetHandleVerifier [0x00007FF7CA7E2A13+1313971]
	GetHandleVerifier [0x00007FF7CA6DDD16+245686]
	(No symbol) [0x00007FF7CA66759F]
	(No symbol) [0x00007FF7CA663814]
	(No symbol) [0x0000

In [10]:


    driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[2]/div[2]/main/div/div[1]/div[2]/div/div/div/div/div/div/div/div/div/label').click()
    print("Changed to Helo2")


Changed to Helo2


In [8]:
driver.find_element(By.XPATH, '//*[@id="menu-list-:r3j:-menuitem-:r3p:"]').click()